# Overall Explanation of Generating Simulated Sequence File 

explanation

# Load Golden Standard Set From 

# Functions

In [19]:
import vcf
import subprocess
import pysam
import pyranges 



def vcf_to_bed(vcf_path, bed_path):
    # Open VCF file
    vcf_reader = vcf.Reader(open(vcf_path, 'r'))

    # Open BED file for writing
    with open(bed_path, 'w') as bed_file:
        # Iterate through VCF records and write to BED file
        for record in vcf_reader:
            chrom = record.CHROM
            start = record.POS - 1  # Convert to 0-based coordinates for BED format
            end = record.POS
            info = record.INFO

            bed_file.write(f"{chrom}\t{start}\t{end}\t{info}\n")


In [20]:
vcf_to_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG002/Alu/HG002.GRCh38.300x_ALU.vcf", "results/xTea/shortread/mosaic/GIAB/Illumina300x/HG002/Alu/HG002.GRCh38.300x_ALU.bed")
vcf_to_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.vcf", "results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.bed")

In [21]:
pyranges.read_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.bed")

/home/jp394/miniforge3/envs/SMaTH/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


,Chromosome,Start,End,Name
0,chr1,20606196,20606197,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 95, 'END': 2..."
1,chr1,23654636,23654637,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 316, 'END': ..."
2,chr1,26904034,26904035,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 452, 'END': ..."
3,chr1,29100312,29100313,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 466, 'END': ..."
4,chr1,37775709,37775710,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 210, 'END': ..."
...,...,...,...,...
271,chr22,46227137,46227138,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 250, 'END': ..."
272,chrX,255569,255570,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 270, 'END': ..."
273,chrX,31982954,31982955,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 265, 'END': ..."
274,chrX,117704900,117704901,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 188, 'END': ..."


In [22]:
pyranges.read_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG002/Alu/HG002.GRCh38.300x_ALU.bed")

/home/jp394/miniforge3/envs/SMaTH/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


,Chromosome,Start,End,Name
0,chr1,21409316,21409317,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 104, 'END': ..."
1,chr1,28994949,28994950,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 286, 'END': ..."
2,chr1,29100312,29100313,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 299, 'END': ..."
3,chr1,37775709,37775710,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 208, 'END': ..."
4,chr1,37967092,37967093,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 296, 'END': ..."
...,...,...,...,...
296,chr22,41314923,41314924,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 270, 'END': ..."
297,chr22,49879715,49879716,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 462, 'END': ..."
298,chrX,34446213,34446214,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 295, 'END': ..."
299,chrX,117704886,117704887,"{'SVTYPE': 'INS:ME:ALU', 'SVLEN': 249, 'END': ..."


# Remove Spikes from BAM File

In [6]:

def remove_spikes(input_bam, bed_file, output_bam):
    try:
        # Open input BAM file
        with pysam.AlignmentFile(input_bam, "rb") as input_bam_file:
            # Open BED file for regions to keep
            with open(bed_file, "r") as bed:
                # Create a list of regions from the BED file
                regions = [line.strip().split('\t') for line in bed]

                # Fetch reads overlapping the specified regions
                filtered_reads = input_bam_file.fetch(regions=regions)

                # Open output BAM file for writing
                with pysam.AlignmentFile(output_bam, "wb", header=input_bam_file.header) as output_bam_file:
                    # Write filtered reads to output BAM file
                    for read in filtered_reads:
                        output_bam_file.write(read)

        print(f"Spikes removed successfully. Output saved to {output_bam}")

    except Exception as e:
        print(f"Error: {e}")

def extract_regions(input_bam, bed_file, output_bam):
    # Step 2: Extract regions from the first BAM file
    with pysam.AlignmentFile(input_bam, "rb") as bam_file:
        regions = pysam.TabixFile(bed_file)
        extracted_reads = [read for read in bam_file.fetch() if regions.fetch(read.reference_name, read.reference_start, read.reference_end)]

    # Write the extracted reads to a new BAM file
    with pysam.AlignmentFile(output_bam, "wb", header=bam_file.header) as output_bam_file:
        for read in extracted_reads:
            output_bam_file.write(read)


def simulate_insertions(reference_genome, extracted_bam, output_prefix):
    try:
        # Open reference genome
        with pysam.FastaFile(reference_genome) as ref_fasta:
            # Open extracted BAM file
            with pysam.AlignmentFile(extracted_bam, "rb") as input_bam_file:
                # Open output BAM file for writing simulated reads
                with pysam.AlignmentFile(output_prefix + ".bam", "wb", header=input_bam_file.header) as output_bam_file:
                    # Iterate over reads in the input BAM file
                    for read in input_bam_file:
                        # Simulate insertions (modify read sequence, qualities, etc. as needed)
                        simulated_read = simulate_insertion(read, ref_fasta)

                        # Write the simulated read to the output BAM file
                        output_bam_file.write(simulated_read)

        print(f"Insertions simulated successfully. Output saved to {output_prefix}.bam")

    except Exception as e:
        print(f"Error: {e}")

def simulate_insertion(read, ref_fasta):
    # Modify the read to simulate an insertion (customize based on your needs)
    # For example, you can extend the read sequence, qualities, etc.
    simulated_read = read.copy()
    simulated_read.query_sequence += "INSERTED_SEQUENCE"
    simulated_read.query_qualities += [30] * len("INSERTED_SEQUENCE")

    # Adjust alignment information if necessary
    simulated_read.reference_end += len("INSERTED_SEQUENCE")

    return simulated_read


def merge_bams(original_bam, simulated_bam, merged_bam):
    # Step 4: Merge BAM files
    with pysam.AlignmentFile(original_bam, "rb") as original_file, \
         pysam.AlignmentFile(simulated_bam + '.sam', "rb") as simulated_file, \
         pysam.AlignmentFile(merged_bam, "wb", header=original_file.header) as output_bam:
        for read in original_file:
            output_bam.write(read)
        for read in simulated_file:
            output_bam.write(read)

    # Step 5: Sort and index merged BAM file
    pysam.sort('-o', merged_bam.replace('.bam', '_sorted.bam'), merged_bam)
    pysam.index(merged_bam.replace('.bam', '_sorted.bam'))

# if __name__ == "__main__":
#     # Specify file paths
#     input_bam = "/path/to/your/original.bam"
#     bed_file = "/path/to/your/simulated_insertions.bed"
#     output_bam = "/path/to/your/extracted_reads.bam"
#     reference_genome = "/path/to/your/reference_genome.fasta"
#     output_prefix = "/path/to/your/simulated_insertions"
#     merged_bam = "/path/to/your/final_merged.bam"

#     # Extract regions from the original BAM file
#     extract_regions(input_bam, bed_file, output_bam)

#     # Simulate insertions
#     simulate_insertions(reference_genome, output_bam, output_prefix)

#     # Merge the original and simulated BAM files
#     merge_bams(input_bam, output_prefix + '.bam', merged_bam)